In [ ]:
import yfinance as yf
import plotly.graph_objects as go


# berkshire hathaway inc. (b stock)
msft = yf.Ticker('MSFT')

hist = msft.history(period="7d", interval="1m", auto_adjust=True)

hist = hist.reset_index()

hist

In [ ]:
fig = go.Figure(data=[go.Candlestick(
                x=hist.index,
                open=hist['Open'],
                high=hist['High'],
                low=hist['Low'],
                close=hist['Close'])])

fig.show()